In [1]:
!curl -C - -o /tmp/DAPI-image.zip http://bigwww.epfl.ch/deconvolution/bio/CElegans-DAPI.zip
!unzip -n /tmp/DAPI-image.zip -d /tmp

!curl -C - -o /tmp/DAPI-psf.zip http://bigwww.epfl.ch/deconvolution/bio/PSF-CElegans-DAPI.zip
!unzip -n /tmp/DAPI-psf.zip -d /tmp

** Resuming transfer from byte position 57231120
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   394    0   394    0     0   1145      0 --:--:-- --:--:-- --:--:--  1148
Archive:  /tmp/DAPI-image.zip
** Resuming transfer from byte position 975540
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   394    0   394    0     0   1219      0 --:--:-- --:--:-- --:--:--  1219
Archive:  /tmp/DAPI-psf.zip


In [2]:
from pathlib import Path

import numpy as np
import cupy as cp
import napari

from tifffile import imread
from numpy.typing import ArrayLike

from dexpv2.deconv import lucy_richardson

In [3]:
def load_img(path: Path) -> ArrayLike:
    files = sorted(path.glob("*.tif"))
    return np.stack([imread(f) for f in files])

img = load_img(Path("/tmp/CElegans-DAPI"))
psf = load_img(Path("/tmp/PSF-CElegans-DAPI"))
img.shape

(104, 712, 672)

In [5]:
n_iters = 10
deconved = lucy_richardson(cp.asarray(img, dtype=np.float32), cp.asarray(psf), n_iters, accelerated=True)

In [6]:
kwargs = {"colormap": "magma", "blending": "additive"}
viewer = napari.Viewer()
viewer.add_image(img, **kwargs)
viewer.add_image(psf, **kwargs)
viewer.add_image(deconved.get(), **kwargs)

<Image layer 'Image' at 0x7f6a980ba290>